## Solução para a Construção do Modelo Preditivo: Câncer de Pulmão

- Enunciado: O Dr. João, renomado pneumologista de João Pessoa, deseja otimizar sua prática médica utilizando análise de dados e inteligência artificial. O objetivo é desenvolver um modelo de Machine Learning capaz de identificar o risco de câncer de pulmão.

- **Objetivo**: Criar um modelo capaz de identificar se os pacientes do Dr. João apresentam o **risco de ter Câncer de Pulmão**.

### 1. Importação das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from imblearn.over_sampling import SMOTE

### 2. Importar Base de Dados

In [72]:
df = pd.read_csv("data/pneumologista_joao_pacientes_bruto.csv")

In [73]:
df

,nome_paciente,idade,sexo,tabagismo,descoloracao_dedos,estresse_mental,exposicao_poluicao,doenca_cronica,nivel_energia,fraqueza_imunologica,...,saturacao_oxigenio,aperto_peito,historico_familiar,historico_tabagismo_familiar,estresse_imunologico,doenca_pulmonar,peso_kg,altura_m,imc,risco_cancer_pulmao
0,Ana Silva,54,Feminino,Não,Não,Baixo,Baixa,Sim,Alto,Não,...,96.7,Não,Não,Sim,Baixo,Não,38.8,1.79,12.1,Sem Risco
1,Mariana Lima,38,Masculino,Não,Não,Moderado,Média,Sim,Moderado,Não,...,NaN,Não,Não,Sim,Moderado,Não,77.8,1.60,30.4,Sem Risco
2,Tiago Barbosa,50,Masculino,Não,Não,Alto,Média,Não,Alto,Não,...,92.1,Sim,Não,Não,Baixo,Não,57.8,1.62,22.0,Sem Risco
3,Carlos Alves,56,Masculino,Não,Não,Baixo,Baixa,Sim,Alto,Não,...,99.2,Não,Não,Não,Alto,Não,83.4,1.64,31.0,Sem Risco
4,Paula Costa,18,Não declarado,Sim,Não,Baixo,Média,Não,Moderado,Não,...,99.1,Não,Sim,Não,Baixo,Não,85.5,1.80,26.4,Risco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,João Nunes,62,Não declarado,Sim,Não,Alto,Média,Não,Moderado,Não,...,99.3,Não,Sim,Não,Baixo,Não,49.9,1.84,14.7,Risco
996,Lucas Santos,43,Feminino,Sim,Não,Moderado,Média,Não,Alto,Não,...,96.7,Não,Não,Não,Baixo,Não,111.0,1.63,41.8,Sem Risco
997,Rafael Pereira,26,Outro,Sim,Não,Baixo,Alta,Não,Moderado,Não,...,96.8,Não,Não,Não,Baixo,Não,90.3,1.81,27.6,Sem Risco
998,Lucas Santos,29,Outro,Sim,Não,Baixo,Alta,Sim,Alto,Não,...,94.3,Não,Não,Sim,Alto,Não,107.5,1.59,42.5,Risco


### 3. Análise de Dados

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   nome_paciente                 1000 non-null   object 
 1   idade                         1000 non-null   int64  
 2   sexo                          1000 non-null   object 
 3   tabagismo                     1000 non-null   object 
 4   descoloracao_dedos            1000 non-null   object 
 5   estresse_mental               1000 non-null   object 
 6   exposicao_poluicao            1000 non-null   object 
 7   doenca_cronica                1000 non-null   object 
 8   nivel_energia                 1000 non-null   object 
 9   fraqueza_imunologica          1000 non-null   object 
 10  problema_respiratorio         1000 non-null   object 
 11  consumo_alcool                1000 non-null   object 
 12  desconforto_garganta          966 non-null    object 
 13  satu

In [95]:
df["risco_cancer_pulmao"].value_counts()

risco_cancer_pulmao
Sem Risco    820
Risco        180
Name: count, dtype: int64

### 4. Tratamento de Dados:

In [75]:
df["desconforto_garganta"] = df["desconforto_garganta"].fillna("Sim")

df["saturacao_oxigenio"] = df["saturacao_oxigenio"].fillna(df["saturacao_oxigenio"].mean())

df["peso_kg"] = df["peso_kg"].fillna(df["peso_kg"].mean())

df["altura_m"] = df["altura_m"].fillna(df["altura_m"].mean())

df["imc"] = df["imc"].fillna(df["imc"].mean())

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   nome_paciente                 1000 non-null   object 
 1   idade                         1000 non-null   int64  
 2   sexo                          1000 non-null   object 
 3   tabagismo                     1000 non-null   object 
 4   descoloracao_dedos            1000 non-null   object 
 5   estresse_mental               1000 non-null   object 
 6   exposicao_poluicao            1000 non-null   object 
 7   doenca_cronica                1000 non-null   object 
 8   nivel_energia                 1000 non-null   object 
 9   fraqueza_imunologica          1000 non-null   object 
 10  problema_respiratorio         1000 non-null   object 
 11  consumo_alcool                1000 non-null   object 
 12  desconforto_garganta          1000 non-null   object 
 13  satu

### 5. Separar a Base de Dados entre Features e Classes:

In [77]:
x_data = df.iloc[:, 0:22].values
y_data = df.iloc[:, 22].values

x_data, y_data

(array([['Ana Silva', 54, 'Feminino', ..., 38.8, 1.79, 12.1],
        ['Mariana Lima', 38, 'Masculino', ..., 77.8, 1.6, 30.4],
        ['Tiago Barbosa', 50, 'Masculino', ..., 57.8, 1.62, 22.0],
        ...,
        ['Rafael Pereira', 26, 'Outro', ..., 90.3, 1.81, 27.6],
        ['Lucas Santos', 29, 'Outro', ..., 107.5, 1.59, 42.5],
        ['Carlos Moreira', 61, 'Não declarado', ..., 74.3, 1.6, 29.0]],
       shape=(1000, 22), dtype=object),
 array(['Sem Risco', 'Sem Risco', 'Sem Risco', 'Sem Risco', 'Risco',
        'Sem Risco', 'Sem Risco', 'Sem Risco', 'Risco', 'Sem Risco',
        'Sem Risco', 'Sem Risco', 'Risco', 'Risco', 'Sem Risco', 'Risco',
        'Sem Risco', 'Sem Risco', 'Sem Risco', 'Sem Risco', 'Sem Risco',
        'Sem Risco', 'Sem Risco', 'Sem Risco', 'Sem Risco', 'Sem Risco',
        'Sem Risco', 'Sem Risco', 'Sem Risco', 'Sem Risco', 'Sem Risco',
        'Sem Risco', 'Sem Risco', 'Sem Risco', 'Sem Risco', 'Risco',
        'Sem Risco', 'Sem Risco', 'Sem Risco', 'Sem Ri

#### 5.1 Segmentar colunas númericas de objetos

In [78]:
colunas_numericas = [index for index in range(x_data.shape[1]) if type(x_data[0][index]) == int or type(x_data[0][index]) == float]
colunas_objetos = [index for index in range(x_data.shape[1]) if type(x_data[0][index]) ==  str or type(x_data[0][index]) == np.str_] 

colunas_numericas, colunas_objetos

([1, 13, 19, 20, 21],
 [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18])

### 6. Aplicando Pré-processamento

In [79]:
encoder_cat = ColumnTransformer(transformers=[('OneHot', OrdinalEncoder(), colunas_objetos)], remainder='passthrough')

scaler = ColumnTransformer(transformers=[('Scaler', StandardScaler(), colunas_numericas)], remainder='passthrough')

preprocessor = Pipeline(steps=[('encoder', encoder_cat),
                                ('scaler', scaler)])

x_data = preprocessor.fit_transform(x_data)

x_data

array([[-1.3429479515630782, -0.3604767715468762, -2.579936784306672,
        ..., 0.0, 54, 96.7],
       [-0.44345166786376267, -0.3604767715468762, -0.03262465889204941,
        ..., 0.0, 38, 96.95772442588726],
       [-0.44345166786376267, -0.3604767715468762, -1.3389385693610865,
        ..., 0.0, 50, 92.1],
       ...,
       [1.3555408995348686, -0.3604767715468762, 0.7838215351510988, ...,
        0.0, 26, 96.8],
       [1.3555408995348686, -0.3604767715468762, 1.907251498154471, ...,
        0.0, 29, 94.3],
       [0.4560446158355529, -0.3604767715468762, -0.2612295932241309,
        ..., 0.0, 61, 97.5]], shape=(1000, 22), dtype=object)

### 7. Tratando Dados Desbalanceados

In [83]:
over_sampling = SMOTE(sampling_strategy="minority")
x_data, y_data = over_sampling.fit_resample(x_data, y_data)

x_data, y_data 

(array([[-1.34294795e+00, -3.60476772e-01, -2.57993678e+00, ...,
          0.00000000e+00,  5.40000000e+01,  9.67000000e+01],
        [-4.43451668e-01, -3.60476772e-01, -3.26246589e-02, ...,
          0.00000000e+00,  3.80000000e+01,  9.69577244e+01],
        [-4.43451668e-01, -3.60476772e-01, -1.33893857e+00, ...,
          0.00000000e+00,  5.00000000e+01,  9.21000000e+01],
        ...,
        [-1.34294795e+00, -3.60476772e-01,  9.49045041e-01, ...,
          3.73266909e-01,  5.06267331e+01,  9.77972271e+01],
        [ 1.58704236e-01, -3.60476772e-01, -1.99200776e-01, ...,
          0.00000000e+00,  4.23388736e+01,  9.83305632e+01],
        [ 4.86477593e-01, -3.60476772e-01, -3.40836360e-01, ...,
          0.00000000e+00,  5.28308332e+01,  9.75661666e+01]],
       shape=(1640, 22)),
 array(['Sem Risco', 'Sem Risco', 'Sem Risco', ..., 'Risco', 'Risco',
        'Risco'], shape=(1640,), dtype=object))

### 8. Dividindo a Base de Dados entre treino e teste

In [84]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3)

### 9. Treinando os Modelos

#### 9.1 Regressão Logística

In [ ]:
modelLogistic = LogisticRegressionCV(solver="liblinear")
modelLogistic.fit(x_train, y_train)
y_pred = modelLogistic.predict(x_test)

In [86]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Risco       0.90      0.94      0.92       235
   Sem Risco       0.94      0.91      0.92       257

    accuracy                           0.92       492
   macro avg       0.92      0.92      0.92       492
weighted avg       0.92      0.92      0.92       492



#### 9.2 SVM (Support Vector Machines)

In [87]:
modelSVC = SVC()
modelSVC.fit(x_train, y_train)
y_pred = modelSVC.predict(x_test)

In [88]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Risco       0.63      0.69      0.66       235
   Sem Risco       0.69      0.63      0.66       257

    accuracy                           0.66       492
   macro avg       0.66      0.66      0.66       492
weighted avg       0.66      0.66      0.66       492



#### 9.3 Random Forest Classifier

In [89]:
modelRFC = RandomForestClassifier()
modelRFC.fit(x_train, y_train)
y_pred = modelRFC.predict(x_test)

In [90]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Risco       0.97      0.91      0.94       235
   Sem Risco       0.92      0.98      0.95       257

    accuracy                           0.94       492
   macro avg       0.95      0.94      0.94       492
weighted avg       0.94      0.94      0.94       492



#### 9.4 Gradient Boosting Classifier

In [91]:
modelGBC = GradientBoostingClassifier()
modelGBC.fit(x_train, y_train)
y_pred = modelGBC.predict(x_test)

In [92]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Risco       0.97      0.91      0.94       235
   Sem Risco       0.92      0.97      0.95       257

    accuracy                           0.94       492
   macro avg       0.95      0.94      0.94       492
weighted avg       0.94      0.94      0.94       492



#### 9.5 KNeighborsClassifier

In [93]:
modelKNN = KNeighborsClassifier()
modelKNN.fit(x_train, y_train)
y_pred = modelKNN.predict(x_test)

In [94]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Risco       0.67      0.91      0.77       235
   Sem Risco       0.87      0.59      0.71       257

    accuracy                           0.74       492
   macro avg       0.77      0.75      0.74       492
weighted avg       0.78      0.74      0.74       492



### 10. Importando o Melhor Modelo

In [ ]:
import joblib 

joblib.dump(modelRFC, "modelRFC.pkl")
joblib.dump(preprocessor, "preprocessor.pkl")